In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchdiffeq import odeint
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import integrate, interpolate
from scipy import optimize
from torch.autograd import grad
from scipy.optimize import curve_fit
from scipy.interpolate import CubicSpline
from scipy.integrate import solve_ivp, odeint
from scipy.interpolate import interp1d

In [2]:
class PINN_sys(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PINN_sys, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size), nn.Tanh(),
            nn.Linear(hidden_size, hidden_size), nn.Tanh(),
            nn.Linear(hidden_size, hidden_size), nn.Tanh(),
            nn.Linear(hidden_size, hidden_size), nn.Tanh(),
            nn.Linear(hidden_size, output_size), nn.Tanh()
        )
        self.params = nn.Parameter(torch.rand(3))  # Learnable parameters

    def forward(self, x):
        return self.net(x)

# Training function
def train_pinn_sys(model, 
                   t_train, 
                   G1_data,
                   G2_data,
                   P_data,
                   D_data,
                   zero_init,
                   known_param,
                   num_epochs, 
                   learning_rate
                   ):
    
    # Define the loss function (MSE for the PINN)
    criterion = nn.MSELoss()

    # Define the optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Define the learning rate scheduler
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=50, factor=0.5, verbose=True)
    
    # write the loss and params to a list
    epoch_list = []
    loss_list = []
    fg_list = []
    fm_list = []
    fp_list = []

    # Training loop
    for epoch in range(num_epochs):
        # Convert to PyTorch tensors
        t_train = torch.FloatTensor(t_train.reshape(-1, 1))
        G1_tensor = torch.FloatTensor(G1_data.reshape(-1, 1))
        G2_tensor = torch.FloatTensor(G2_data.reshape(-1, 1))
        P_tensor = torch.FloatTensor(P_data.reshape(-1, 1))
        D_tensor = torch.FloatTensor(D_data.reshape(-1, 1))
        zero_init = torch.FloatTensor(zero_init.reshape(-1, 1))
        known_param = torch.FloatTensor(known_param.reshape(-1, 1))
        
        t_train.requires_grad_()

        # Forward pass
        NN = model(t_train)
        G1, G2, P, D = torch.split(NN, 1, dim=1)
        
        G1_ones = torch.ones_like(G1)
        G2_ones = torch.ones_like(G2)
        P_ones = torch.ones_like(P)
        D_ones = torch.ones_like(D)

        # Compute the gradients of the variables
        G1_t = grad(G1, t_train, G1_ones, create_graph=True)[0]
        G2_t = grad(G2, t_train, G2_ones, create_graph=True)[0]
        P_t = grad(P, t_train, P_ones, create_graph=True)[0]
        D_t = grad(D, t_train, D_ones, create_graph=True)[0]
        
        # ensure the model parameters are positive
        param_reg = torch.abs(model.params)
        fg = param_reg[0]
        fm = param_reg[1]
        fp = param_reg[2]
        
        k1 = known_param[0]
        k2 = known_param[1]
        k3 = known_param[2]
        kd = known_param[3]
        kd2 = known_param[4]
        cGEM = known_param[5]
        
        # residual loss
        l1 = criterion(G1_t, -k1*(1-fg)*G1 + 2*k2*(1-fm)*G2 - kd*G1)
        l2 = criterion(G2_t, k1*(1-fg)*G1 - k2*(1-fm)*G2 - k3*(1-fp)*G2 - kd*cGEM*G2)
        l3 = criterion(P_t, k3*(1-fp)*G2 - kd2*P)
        l4 = criterion(D_t, kd*G1 + kd*cGEM*G2 + kd2*P)
        
        # data loss
        sum_loss1 = criterion(G1, G1_tensor)
        sum_loss2 = criterion(G2, G2_tensor)
        sum_loss3 = criterion(P, P_tensor)
        sum_loss4 = criterion(D, D_tensor)
        
        # initial conditions loss
        G1_init_loss = criterion(G1[0], zero_init[0])
        G2_init_loss = criterion(G2[0], zero_init[1])
        P_init_loss = criterion(P[0], zero_init[2])
        D_init_loss = criterion(D[0], zero_init[3])
        
       
        # residual loss + data loss + initial conditions loss 
        total_loss = (l1 + l2 + l3 + l4 + \
                      sum_loss1 + sum_loss2 + sum_loss3 + sum_loss4 +  \
                      G1_init_loss + G2_init_loss + P_init_loss + D_init_loss)       

        # Backward pass and optimization
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        # Update the learning rate based on the validation loss
        scheduler.step(total_loss)
        
        # Print progress
        if (epoch + 1) % 200 == 0:
            loss_item = round(total_loss.item(), 4)
            param_item = param_reg.data.numpy()
            epoch_list.append(epoch+1)
            loss_list.append(loss_item)
            fg_list.append(param_item[0])
            fm_list.append(param_item[1])
            fp_list.append(param_item[2])
            print(f'Epoch [{epoch+1}/{num_epochs}]', 
                  f'Loss: {total_loss.item():.2e}', 
                  f'params: {param_reg.data.numpy()}')
            
        # save list containing loss and params to csv files
        loss_param_df = pd.DataFrame(data={"Num Epoch": epoch_list,
                                           "loss": loss_list,
                                           "fg": fg_list,
                                           "fm": fm_list,
                                           "fp": fp_list
                                           })
        loss_param_df.to_csv("./loss_param_1e.csv", sep=',',index=False)

    print('Training completed.')

In [3]:
t = np.linspace(2,39,38)

print(t)

[ 2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18. 19.
 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37.
 38. 39.]


In [4]:

def drug_dose_param(file,known_param,t):
    #file = pd.read_csv('filename')
    
    file_D = file.iloc[0,:-1].to_numpy()
    file_G1 = file.iloc[1,:-1].to_numpy()
    file_G2 = file.iloc[2,:-1].to_numpy()
    file_P = file.iloc[3,:-1].to_numpy()
    
    # scaling the data
    max_file_G1 = np.max(file_G1)
    max_file_G2 = np.max(file_G2)
    max_file_P = np.max(file_P)
    max_file_D = np.max(file_D)
    
    max_file = np.max([max_file_G1, max_file_G2, max_file_P, max_file_D])
    ###########################################################################
    
    t_train = t
    file_G1_data = file_G1 / max_file
    file_G2_data = file_G2 / max_file
    file_P_data = file_P / max_file
    file_D_data = file_D / max_file
    
    # Set initial conditions for all variables
    file_zero_init = np.array([file_G1_data[0], file_G2_data[0], file_P_data[0], file_D_data[0]])
    
    # Create and train the PINN model
    input_size = 1; hidden_size = 32; output_size = 4; num_epochs =  6000; learning_rate = 1e-3
    
    model = PINN_sys(input_size, hidden_size, output_size)
    train_pinn_sys(model,
                   t_train, file_G1_data, file_G2_data, file_P_data,
                   file_D_data, file_zero_init, known_param, num_epochs, learning_rate)
    
    # Test the trained model
    t_test = t
    t_test = torch.FloatTensor(t_test.reshape(-1, 1))
    file_pred = model(t_test).detach().numpy()
    
    file_G1_pred = file_pred[:, 0] * max_file
    file_G2_pred = file_pred[:, 1] * max_file
    file_P_pred = file_pred[:, 2] * max_file
    file_D_pred = file_pred[:, 3] * max_file
    
    return file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred


In [5]:
cGEM_all = np.array([0, 3.125, 6.25, 12.5, 25, 50, 100, 200, 400, 800])

kn_param_2N = np.array([0.06542416, 0.65263357, 0.05263728, 0.01360368, 0.00667656])
kn_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785])

In [6]:
# GEM 4N
# E_3, cGEM=3.125

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 3.125])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/E_3.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 2.95e-03 params: [0.7900826  0.55716574 0.55686635]
Epoch [400/6000] Loss: 1.30e-03 params: [0.669077   0.46709782 0.73614156]
Epoch [600/6000] Loss: 1.01e-03 params: [0.547382   0.38135287 0.8968517 ]
Epoch [800/6000] Loss: 9.20e-04 params: [0.44224542 0.30816686 1.0353366 ]
Epoch [1000/6000] Loss: 1.00e-03 params: [0.35432494 0.2446183  1.1498189 ]
Epoch [1200/6000] Loss: 8.14e-04 params: [0.28138453 0.18585572 1.2387947 ]
Epoch [1400/6000] Loss: 7.87e-04 params: [0.22403896 0.13957407 1.3035587 ]
Epoch 01518: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1600/6000] Loss: 7.72e-04 params: [0.18772925 0.11188693 1.338713  ]
Epoch [1800/6000] Loss: 7.63e-04 params: [0.16754456 0.09171098 1.3529894 ]
Epoch [2000/6000] Loss: 7.55e-04 params: [0.14726289 0.0729106  1.3614918 ]
Epoch [2200/6000] Loss: 7.48e-04 params: [0.12784469 0.05539685 1.3627377 ]
Epoch [2400/6000] Loss: 7.43e-04 params: [0.1098368  0.03920526 1.3555976 ]
Epoch [2600/6000] Loss: 7.39e-

In [7]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.048942767
fm=3.3381486e-05
fp=0.93288296


In [8]:
# GEM 2N
# F_3, cGEM=3.125

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 3.125])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/F_3.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 6.19e-03 params: [0.91036344 0.84215    0.82518077]
Epoch [400/6000] Loss: 2.14e-03 params: [0.89859086 0.7593382  1.0601972 ]
Epoch [600/6000] Loss: 2.63e-03 params: [0.80131775 0.6994678  1.2705611 ]
Epoch [800/6000] Loss: 1.31e-03 params: [0.68327594 0.63800454 1.4461793 ]
Epoch [1000/6000] Loss: 1.21e-03 params: [0.56349224 0.57730997 1.5888922 ]
Epoch 01077: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1200/6000] Loss: 1.17e-03 params: [0.48352373 0.5369295  1.668503  ]
Epoch [1400/6000] Loss: 1.14e-03 params: [0.4267149 0.5077612 1.7168474]
Epoch [1600/6000] Loss: 1.11e-03 params: [0.3707694  0.47884113 1.7565753 ]
Epoch [1800/6000] Loss: 1.08e-03 params: [0.31659526 0.4507524  1.7858624 ]
Epoch [2000/6000] Loss: 1.06e-03 params: [0.26497436 0.42392677 1.8028312 ]
Epoch [2200/6000] Loss: 1.05e-03 params: [0.21653275 0.39882547 1.8057011 ]
Epoch [2400/6000] Loss: 1.06e-03 params: [0.17184214 0.37569228 1.7929025 ]
Epoch [2600/6000] Loss: 1.02e-03 

In [9]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=5.8532105e-06
fm=0.28474462
fp=1.0649403


In [10]:
# GEM 2N
# G_3, cGEM=3.125

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 3.125])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/G_3.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 4.89e-03 params: [0.64719075 0.7150366  0.3500431 ]
Epoch [400/6000] Loss: 2.13e-03 params: [0.7190191  0.63528603 0.46530282]
Epoch [600/6000] Loss: 1.62e-03 params: [0.7035904  0.58108157 0.6383437 ]
Epoch 00757: reducing learning rate of group 0 to 5.0000e-04.
Epoch [800/6000] Loss: 1.46e-03 params: [0.65507454 0.53899103 0.79540837]
Epoch [1000/6000] Loss: 1.38e-03 params: [0.62060285 0.51495224 0.87977415]
Epoch [1200/6000] Loss: 1.32e-03 params: [0.5823773 0.4895687 0.9612776]
Epoch [1400/6000] Loss: 1.26e-03 params: [0.5418113 0.4631621 1.0389299]
Epoch [1600/6000] Loss: 1.21e-03 params: [0.4999792 0.4360487 1.111968 ]
Epoch [1800/6000] Loss: 1.18e-03 params: [0.4576593  0.40857035 1.1797365 ]
Epoch [2000/6000] Loss: 1.15e-03 params: [0.4155669 0.3810261 1.2415457]
Epoch [2200/6000] Loss: 1.13e-03 params: [0.37421328 0.35360873 1.2966571 ]
Epoch [2400/6000] Loss: 2.64e-03 params: [0.3339743  0.32640508 1.3441639 ]
Epoch [2600/6000] Loss: 1.09e-03 params: [

In [11]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=1.2257206e-06
fm=0.068456426
fp=0.9525176


In [12]:
# GEM 2N
# H_3, cGEM=3.125

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 3.125])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/H_3.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 7.18e-03 params: [0.8699785  0.55540156 0.7008266 ]
Epoch [400/6000] Loss: 3.03e-03 params: [0.8104594  0.55200076 0.73886025]
Epoch [600/6000] Loss: 2.25e-03 params: [0.7638028  0.5413792  0.78623724]
Epoch [800/6000] Loss: 1.90e-03 params: [0.72501737 0.5248829  0.8397498 ]
Epoch [1000/6000] Loss: 1.62e-03 params: [0.6883122  0.50418043 0.89848804]
Epoch 01019: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1200/6000] Loss: 1.49e-03 params: [0.66722876 0.49113604 0.9334135 ]
Epoch [1400/6000] Loss: 1.41e-03 params: [0.6466611  0.47767216 0.9681043 ]
Epoch [1600/6000] Loss: 1.34e-03 params: [0.6246537 0.4627943 1.005194 ]
Epoch [1800/6000] Loss: 1.28e-03 params: [0.6012313  0.44661033 1.0442592 ]
Epoch [2000/6000] Loss: 1.24e-03 params: [0.5765022  0.42925745 1.0848682 ]
Epoch [2200/6000] Loss: 1.25e-03 params: [0.55057037 0.41083553 1.1265827 ]
Epoch 02206: reducing learning rate of group 0 to 2.5000e-04.
Epoch [2400/6000] Loss: 1.18e-03 params: [0.536

In [13]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.21821547
fm=0.14673647
fp=1.4883664


In [14]:
# GEM 2N
# E_4, cGEM=6.25

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 6.25])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/E_4.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 8.65e-03 params: [0.50654876 0.46612468 0.95661294]
Epoch [400/6000] Loss: 3.65e-03 params: [0.3539896 0.5539567 1.1263076]
Epoch [600/6000] Loss: 3.27e-03 params: [0.2732552 0.5785435 1.2647848]
Epoch [800/6000] Loss: 2.22e-03 params: [0.22508839 0.570469   1.3843732 ]
Epoch [1000/6000] Loss: 2.01e-03 params: [0.1887965 0.5615072 1.498878 ]
Epoch [1200/6000] Loss: 1.89e-03 params: [0.16626857 0.5586441  1.6126375 ]
Epoch 01298: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1400/6000] Loss: 1.83e-03 params: [0.16096406 0.5609481  1.6999015 ]
Epoch [1600/6000] Loss: 1.80e-03 params: [0.16340415 0.56384146 1.7625439 ]
Epoch [1800/6000] Loss: 1.77e-03 params: [0.17074579 0.56807214 1.8299055 ]
Epoch [2000/6000] Loss: 1.74e-03 params: [0.18271402 0.5735494  1.9024477 ]
Epoch [2200/6000] Loss: 1.72e-03 params: [0.1989736 0.5801408 1.9805183]
Epoch [2400/6000] Loss: 1.70e-03 params: [0.21892889 0.5876233  2.0642133 ]
Epoch 02490: reducing learning rate of gro

In [15]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.4589899
fm=0.66858166
fp=2.8191557


In [16]:
# GEM 2N
# F_4, cGEM=6.25

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 6.25])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/F_4.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 7.40e-03 params: [0.23207232 0.29486787 0.3389863 ]
Epoch [400/6000] Loss: 4.11e-03 params: [0.05155976 0.4348282  0.49423283]
Epoch [600/6000] Loss: 2.58e-03 params: [3.8245311e-05 5.6002265e-01 6.2848109e-01]
Epoch [800/6000] Loss: 2.33e-03 params: [3.0011999e-05 6.1894697e-01 7.2445810e-01]
Epoch [1000/6000] Loss: 2.19e-03 params: [1.6837857e-06 6.3132846e-01 7.9398698e-01]
Epoch [1200/6000] Loss: 2.10e-03 params: [7.7209393e-07 6.3416630e-01 8.6070514e-01]
Epoch [1400/6000] Loss: 2.04e-03 params: [3.0799777e-07 6.3629919e-01 9.3042845e-01]
Epoch [1600/6000] Loss: 2.00e-03 params: [3.6225831e-06 6.3838667e-01 1.0033871e+00]
Epoch [1800/6000] Loss: 1.96e-03 params: [3.9661629e-04 6.3907862e-01 1.0780467e+00]
Epoch 01868: reducing learning rate of group 0 to 5.0000e-04.
Epoch [2000/6000] Loss: 1.95e-03 params: [0.00288865 0.6406101  1.1306925 ]
Epoch [2200/6000] Loss: 1.94e-03 params: [0.00616376 0.6413444  1.1722966 ]
Epoch [2400/6000] Loss: 1.93e-03 params: [0

In [17]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.13935164
fm=0.67333984
fp=2.0293932


In [18]:
# GEM 2N
# G_4, cGEM=6.25

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 6.25])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/G_4.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 6.48e-03 params: [0.6609618  0.6503072  0.94132113]
Epoch [400/6000] Loss: 2.26e-03 params: [0.52967525 0.53618556 1.1308647 ]
Epoch [600/6000] Loss: 1.84e-03 params: [0.36244768 0.44304913 1.2751057 ]
Epoch [800/6000] Loss: 1.62e-03 params: [0.22133322 0.36815974 1.3640782 ]
Epoch [1000/6000] Loss: 1.50e-03 params: [0.11239699 0.31138977 1.3965391 ]
Epoch 01074: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1200/6000] Loss: 1.45e-03 params: [0.05491378 0.28252387 1.3855668 ]
Epoch [1400/6000] Loss: 1.40e-03 params: [0.01943148 0.2633943  1.358479  ]
Epoch [1600/6000] Loss: 1.36e-03 params: [1.2324166e-05 2.4693964e-01 1.3150966e+00]
Epoch [1800/6000] Loss: 1.33e-03 params: [2.7438762e-05 2.3947307e-01 1.2675300e+00]
Epoch [2000/6000] Loss: 1.30e-03 params: [1.9075122e-05 2.3686646e-01 1.2174305e+00]
Epoch [2200/6000] Loss: 1.28e-03 params: [1.3222895e-05 2.3654036e-01 1.1644006e+00]
Epoch [2400/6000] Loss: 1.27e-03 params: [2.3259439e-05 2.3629908e-01 

In [19]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=2.0507825e-06
fm=0.23787631
fp=0.6392054


In [20]:
# GEM 2N
# H_4, cGEM=6.25

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 6.25])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/H_4.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 6.72e-03 params: [0.02408314 0.35485092 0.56785923]
Epoch [400/6000] Loss: 1.95e-03 params: [0.06352835 0.34804818 0.57444584]
Epoch [600/6000] Loss: 1.61e-03 params: [0.10727253 0.33976546 0.5864912 ]
Epoch [800/6000] Loss: 1.34e-03 params: [0.13827239 0.3313536  0.6048834 ]
Epoch [1000/6000] Loss: 1.24e-03 params: [0.15391527 0.32314444 0.62948704]
Epoch 01151: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1200/6000] Loss: 1.18e-03 params: [0.15767936 0.3159979  0.65553564]
Epoch [1400/6000] Loss: 1.13e-03 params: [0.15629737 0.31165704 0.6726323 ]
Epoch [1600/6000] Loss: 1.09e-03 params: [0.15324453 0.3069469  0.6917439 ]
Epoch [1800/6000] Loss: 1.06e-03 params: [0.14850186 0.30185568 0.71275544]
Epoch [2000/6000] Loss: 1.03e-03 params: [0.14227493 0.29636902 0.7355268 ]
Epoch [2200/6000] Loss: 1.20e-03 params: [0.13497694 0.2905008  0.75992775]
Epoch [2400/6000] Loss: 9.81e-04 params: [0.12640938 0.28423354 0.7858284 ]
Epoch 02472: reducing learning

In [21]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.008587429
fm=0.20006588
fp=1.0710055


In [22]:
# GEM 2N
# E_5, cGEM=12.5

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 12.5])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/E_5.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 2.21e-02 params: [0.04167582 0.13234228 0.7279554 ]
Epoch [400/6000] Loss: 1.55e-02 params: [7.5442760e-05 1.6092917e-01 7.5886238e-01]
Epoch [600/6000] Loss: 1.46e-02 params: [7.105259e-05 2.026836e-01 8.045382e-01]
Epoch [800/6000] Loss: 1.34e-02 params: [1.5406906e-04 2.5857297e-01 8.6615521e-01]
Epoch [1000/6000] Loss: 1.22e-02 params: [9.7901153e-05 3.3144996e-01 9.4721591e-01]
Epoch [1200/6000] Loss: 1.07e-02 params: [1.8581351e-05 4.2569041e-01 1.0534973e+00]
Epoch [1400/6000] Loss: 8.76e-03 params: [1.0879032e-04 5.4464400e-01 1.1912340e+00]
Epoch [1600/6000] Loss: 6.17e-03 params: [7.1010756e-05 6.7795599e-01 1.3563069e+00]
Epoch [1800/6000] Loss: 4.75e-03 params: [7.8761732e-05 7.7880228e-01 1.5111173e+00]
Epoch [2000/6000] Loss: 4.27e-03 params: [9.4737425e-06 8.1813985e-01 1.6266363e+00]
Epoch [2200/6000] Loss: 3.90e-03 params: [1.0951964e-05 8.2906258e-01 1.7262198e+00]
Epoch [2400/6000] Loss: 3.64e-03 params: [5.0278140e-07 8.3515066e-01 1.8304974e+

In [23]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.27072486
fm=0.86827725
fp=3.2862406


In [24]:
# GEM 2N
# F_5, cGEM=12.5

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 12.5])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/F_5.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 1.96e-02 params: [0.5473143  0.80745435 0.4632913 ]
Epoch [400/6000] Loss: 6.86e-03 params: [0.42729315 0.92506826 0.65451014]
Epoch [600/6000] Loss: 5.71e-03 params: [0.37921104 0.936849   0.79976875]
Epoch [800/6000] Loss: 4.89e-03 params: [0.33508804 0.9327028  0.942551  ]
Epoch [1000/6000] Loss: 4.60e-03 params: [0.28453553 0.92813003 1.0852208 ]
Epoch [1200/6000] Loss: 4.41e-03 params: [0.22842109 0.92351174 1.2247622 ]
Epoch [1400/6000] Loss: 4.26e-03 params: [0.1677595  0.91886896 1.3588872 ]
Epoch [1600/6000] Loss: 4.08e-03 params: [0.1035912 0.9138877 1.4855744]
Epoch [1800/6000] Loss: 3.91e-03 params: [0.03659626 0.9092439  1.603386  ]
Epoch [2000/6000] Loss: 3.80e-03 params: [2.3530987e-05 9.0624446e-01 1.7120862e+00]
Epoch [2200/6000] Loss: 4.34e-03 params: [6.4840118e-05 9.0600836e-01 1.8169913e+00]
Epoch 02212: reducing learning rate of group 0 to 5.0000e-04.
Epoch [2400/6000] Loss: 3.71e-03 params: [3.0465015e-05 9.0586644e-01 1.8723667e+00]
Epoch 

In [25]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=1.540815e-06
fm=0.9034098
fp=2.5255768


In [26]:
# GEM 2N
# G_5, cGEM=12.5

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 12.5])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/G_5.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 1.08e-02 params: [0.45947844 0.53860235 0.30927166]
Epoch [400/6000] Loss: 4.96e-03 params: [0.28009066 0.75177723 0.5590946 ]
Epoch [600/6000] Loss: 3.87e-03 params: [0.19658825 0.8368631  0.7521486 ]
Epoch [800/6000] Loss: 3.64e-03 params: [0.15753292 0.8408513  0.909965  ]
Epoch [1000/6000] Loss: 3.39e-03 params: [0.12089282 0.838424   1.0637769 ]
Epoch [1200/6000] Loss: 3.17e-03 params: [0.0865568 0.837476  1.2154157]
Epoch [1400/6000] Loss: 2.96e-03 params: [0.05803441 0.83744305 1.3637774 ]
Epoch [1600/6000] Loss: 2.87e-03 params: [0.0353985 0.8364472 1.5064253]
Epoch 01775: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1800/6000] Loss: 2.83e-03 params: [0.0168425 0.8347883 1.6338304]
Epoch [2000/6000] Loss: 2.80e-03 params: [0.0074019 0.8329881 1.6996857]
Epoch [2200/6000] Loss: 2.78e-03 params: [8.606814e-06 8.316971e-01 1.765621e+00]
Epoch [2400/6000] Loss: 2.77e-03 params: [6.8779691e-06 8.3143604e-01 1.8319025e+00]
Epoch [2600/6000] Loss: 2.7

In [27]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=5.503153e-06
fm=0.8298938
fp=2.6088068


In [28]:
# GEM 2N
# H_5, cGEM=12.5

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 12.5])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/H_5.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 3.65e-02 params: [6.1462415e-06 4.8797887e-02 5.0217044e-01]
Epoch [400/6000] Loss: 2.66e-02 params: [2.1042334e-04 1.1982346e-01 5.6850272e-01]
Epoch [600/6000] Loss: 2.39e-02 params: [4.8872156e-05 2.2880015e-01 6.7441577e-01]
Epoch [800/6000] Loss: 2.01e-02 params: [6.404288e-05 3.876549e-01 8.344604e-01]
Epoch [1000/6000] Loss: 1.28e-02 params: [1.8519958e-04 6.1164606e-01 1.0722606e+00]
Epoch [1200/6000] Loss: 5.59e-03 params: [9.4082134e-07 8.2833058e-01 1.3383409e+00]
Epoch [1400/6000] Loss: 4.85e-03 params: [2.1424288e-05 8.8694662e-01 1.4961666e+00]
Epoch [1600/6000] Loss: 4.77e-03 params: [1.267912e-05 8.910900e-01 1.613017e+00]
Epoch [1800/6000] Loss: 4.69e-03 params: [1.0393574e-05 8.9110458e-01 1.7240778e+00]
Epoch [2000/6000] Loss: 4.61e-03 params: [3.2858967e-05 8.9103109e-01 1.8310319e+00]
Epoch 02105: reducing learning rate of group 0 to 5.0000e-04.
Epoch [2200/6000] Loss: 4.57e-03 params: [1.061211e-05 8.910243e-01 1.909849e+00]
Epoch [2400/6000

In [29]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=2.677384e-05
fm=0.89286816
fp=2.7416446


In [30]:
# GEM 2N
# E_6, cGEM=25

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 25])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/E_6.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 2.60e-01 params: [0.01278264 0.41723514 0.8060002 ]
Epoch [400/6000] Loss: 1.40e-01 params: [2.4258377e-04 7.1132392e-01 1.1124709e+00]
Epoch [600/6000] Loss: 4.66e-02 params: [5.9364131e-05 9.5390016e-01 1.4157515e+00]
Epoch [800/6000] Loss: 4.00e-02 params: [2.7057071e-05 1.0068508e+00 1.6050955e+00]
Epoch [1000/6000] Loss: 3.91e-02 params: [6.9639086e-06 1.0100096e+00 1.7678770e+00]
Epoch [1200/6000] Loss: 3.84e-02 params: [7.2510356e-06 1.0094671e+00 1.9289615e+00]
Epoch [1400/6000] Loss: 3.79e-02 params: [9.2229393e-06 1.0087794e+00 2.0885849e+00]
Epoch [1600/6000] Loss: 3.77e-02 params: [9.7063376e-06 1.0080571e+00 2.2456865e+00]
Epoch [1800/6000] Loss: 3.69e-02 params: [1.1692493e-05 1.0073761e+00 2.3996389e+00]
Epoch [2000/6000] Loss: 3.65e-02 params: [1.0708550e-05 1.0067148e+00 2.5499184e+00]
Epoch [2200/6000] Loss: 3.62e-02 params: [1.1031559e-05 1.0060965e+00 2.6961658e+00]
Epoch [2400/6000] Loss: 3.58e-02 params: [1.1596514e-05 1.0054555e+00 2.838028

In [31]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=3.340596e-05
fm=1.000728
fp=4.280553


In [32]:
# GEM 2N
# F_6, cGEM=25

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 25])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/F_6.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 2.04e-01 params: [0.51530486 0.6002498  1.1199065 ]
Epoch [400/6000] Loss: 6.73e-02 params: [0.3161297 0.8599209 1.4006827]
Epoch [600/6000] Loss: 2.83e-02 params: [0.21934637 0.993567   1.6211469 ]
Epoch [800/6000] Loss: 2.67e-02 params: [0.18380126 1.0116632  1.7782108 ]
Epoch [1000/6000] Loss: 2.61e-02 params: [0.15390912 1.0114783  1.9289938 ]
Epoch [1200/6000] Loss: 2.56e-02 params: [0.12074645 1.0102053  2.0802126 ]
Epoch [1400/6000] Loss: 2.51e-02 params: [0.08388893 1.0088958  2.230506  ]
Epoch [1600/6000] Loss: 2.46e-02 params: [0.04344428 1.0075082  2.378536  ]
Epoch [1800/6000] Loss: 2.43e-02 params: [4.0323375e-04 1.0060824e+00 2.5233657e+00]
Epoch [2000/6000] Loss: 2.40e-02 params: [7.9078789e-05 1.0053581e+00 2.6649404e+00]
Epoch [2200/6000] Loss: 2.37e-02 params: [7.0804774e-05 1.0046997e+00 2.8029888e+00]
Epoch [2400/6000] Loss: 2.35e-02 params: [8.6578730e-06 1.0040834e+00 2.9370563e+00]
Epoch [2600/6000] Loss: 2.34e-02 params: [5.5202487e-05 1.0

In [33]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=1.6821685e-05
fm=0.9984005
fp=4.1156373


In [34]:
# GEM 2N
# G_6, cGEM=25

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 25])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/G_6.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 2.20e-01 params: [0.77453655 0.545974   0.30683607]
Epoch [400/6000] Loss: 9.12e-02 params: [0.5610074  0.8168991  0.60084724]
Epoch [600/6000] Loss: 3.28e-02 params: [0.43990174 0.98672044 0.86178017]
Epoch [800/6000] Loss: 2.93e-02 params: [0.39535668 1.0137042  1.0483978 ]
Epoch [1000/6000] Loss: 3.08e-02 params: [0.36033812 1.0136328  1.2223904 ]
Epoch [1200/6000] Loss: 2.77e-02 params: [0.32181305 1.011876   1.3945488 ]
Epoch [1400/6000] Loss: 2.71e-02 params: [0.2790204 1.0100943 1.5645015]
Epoch [1600/6000] Loss: 2.65e-02 params: [0.23206022 1.0082338  1.7316097 ]
Epoch [1800/6000] Loss: 2.59e-02 params: [0.18100232 1.0063639  1.8955582 ]
Epoch [2000/6000] Loss: 2.54e-02 params: [0.12598056 1.0044143  2.0560303 ]
Epoch [2200/6000] Loss: 2.48e-02 params: [0.06705938 1.00246    2.2128415 ]
Epoch [2400/6000] Loss: 2.42e-02 params: [0.00427518 1.0005296  2.3657653 ]
Epoch [2600/6000] Loss: 2.35e-02 params: [1.2642919e-05 9.9993318e-01 2.5154197e+00]
Epoch [280

In [35]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=1.4426978e-05
fm=0.99280643
fp=4.2057157


In [36]:
# GEM 2N
# H_6, cGEM=25

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 25])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/H_6.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 4.04e-02 params: [0.74051964 1.0297912  0.8811191 ]
Epoch [400/6000] Loss: 3.23e-02 params: [0.6556907 1.0348219 1.0631082]
Epoch [600/6000] Loss: 3.02e-02 params: [0.55690366 1.0322517  1.2419655 ]
Epoch [800/6000] Loss: 2.88e-02 params: [0.44598487 1.0297877  1.4186814 ]
Epoch [1000/6000] Loss: 2.73e-02 params: [0.32748926 1.0275143  1.5924528 ]
Epoch [1200/6000] Loss: 2.61e-02 params: [0.20496878 1.0252448  1.7624133 ]
Epoch [1400/6000] Loss: 2.52e-02 params: [0.08042047 1.0228736  1.927939  ]
Epoch [1600/6000] Loss: 2.45e-02 params: [8.7480075e-05 1.0207733e+00 2.0887012e+00]
Epoch [1800/6000] Loss: 2.40e-02 params: [2.2080530e-05 1.0201137e+00 2.2461174e+00]
Epoch [2000/6000] Loss: 2.36e-02 params: [9.3788396e-05 1.0192676e+00 2.4003820e+00]
Epoch [2200/6000] Loss: 2.31e-02 params: [2.1679562e-05 1.0186949e+00 2.5512908e+00]
Epoch [2400/6000] Loss: 2.27e-02 params: [1.1138502e-04 1.0180495e+00 2.6985226e+00]
Epoch [2600/6000] Loss: 2.24e-02 params: [2.042273

In [37]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=2.6213222e-05
fm=1.0128875
fp=4.0853634


In [38]:
# GEM 2N
# E_7, cGEM=50

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 50])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/E_7.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 7.28e-02 params: [0.37199715 0.5609745  0.26468846]
Epoch [400/6000] Loss: 4.21e-02 params: [0.16966021 0.79411334 0.5210064 ]
Epoch [600/6000] Loss: 2.38e-02 params: [0.02390854 0.9882431  0.80396706]
Epoch [800/6000] Loss: 2.18e-02 params: [1.0483855e-04 1.0448000e+00 1.0276024e+00]
Epoch [1000/6000] Loss: 2.13e-02 params: [2.3958357e-05 1.0515875e+00 1.2270241e+00]
Epoch [1200/6000] Loss: 2.09e-02 params: [1.7350518e-05 1.0513569e+00 1.4207941e+00]
Epoch [1400/6000] Loss: 2.03e-02 params: [2.5832238e-05 1.0505730e+00 1.6117036e+00]
Epoch [1600/6000] Loss: 1.98e-02 params: [2.0126568e-05 1.0498182e+00 1.8003110e+00]
Epoch [1800/6000] Loss: 1.94e-02 params: [3.5414992e-05 1.0490968e+00 1.9869401e+00]
Epoch [2000/6000] Loss: 1.91e-02 params: [4.2215881e-05 1.0482562e+00 2.1717470e+00]
Epoch [2200/6000] Loss: 1.88e-02 params: [1.2055782e-04 1.0474027e+00 2.3547704e+00]
Epoch [2400/6000] Loss: 1.86e-02 params: [1.0352135e-04 1.0465649e+00 2.5360258e+00]
Epoch [2600

In [39]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=4.723844e-05
fm=1.034263
fp=5.092544


In [40]:
# GEM 2N
# F_7, cGEM=50

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 50])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/F_7.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 1.48e-01 params: [0.5316796  0.31286627 0.87604994]
Epoch [400/6000] Loss: 1.24e-01 params: [0.3182947  0.49311498 1.0572278 ]
Epoch [600/6000] Loss: 7.44e-02 params: [0.10442103 0.77763855 1.3702241 ]
Epoch [800/6000] Loss: 3.82e-02 params: [3.5585286e-05 1.0070928e+00 1.7060941e+00]
Epoch [1000/6000] Loss: 3.57e-02 params: [2.4566571e-05 1.0570539e+00 1.9437615e+00]
Epoch [1200/6000] Loss: 3.53e-02 params: [1.1321914e-05 1.0607886e+00 2.1500318e+00]
Epoch [1400/6000] Loss: 3.49e-02 params: [5.2392905e-05 1.0601507e+00 2.3478153e+00]
Epoch [1600/6000] Loss: 3.45e-02 params: [1.9275463e-05 1.0592864e+00 2.5402946e+00]
Epoch [1800/6000] Loss: 3.42e-02 params: [2.1183685e-05 1.0583823e+00 2.7284584e+00]
Epoch [2000/6000] Loss: 3.38e-02 params: [1.9780091e-06 1.0575180e+00 2.9129999e+00]
Epoch [2200/6000] Loss: 3.38e-02 params: [4.8693157e-05 1.0566734e+00 3.0943327e+00]
Epoch [2400/6000] Loss: 3.32e-02 params: [3.2605236e-05 1.0558378e+00 3.2727342e+00]
Epoch [2600

In [41]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=2.488275e-05
fm=1.0428274
fp=6.0146546


In [42]:
# GEM 2N
# G_7, cGEM=50

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 50])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/G_7.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 8.19e-02 params: [0.35152915 0.8225233  1.2050297 ]
Epoch [400/6000] Loss: 4.27e-02 params: [0.19682695 1.0046039  1.44987   ]
Epoch [600/6000] Loss: 3.89e-02 params: [0.1272147 1.058293  1.6486465]
Epoch [800/6000] Loss: 3.78e-02 params: [0.07914238 1.0633522  1.830961  ]
Epoch [1000/6000] Loss: 3.72e-02 params: [0.02827492 1.0619664  2.012127  ]
Epoch [1200/6000] Loss: 3.67e-02 params: [3.0375259e-06 1.0603237e+00 2.1931117e+00]
Epoch [1400/6000] Loss: 3.77e-02 params: [6.5480650e-05 1.0594738e+00 2.3738282e+00]
Epoch [1600/6000] Loss: 3.59e-02 params: [6.2950727e-05 1.0587037e+00 2.5536737e+00]
Epoch 01716: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1800/6000] Loss: 3.57e-02 params: [2.5516592e-06 1.0581396e+00 2.6955101e+00]
Epoch [2000/6000] Loss: 3.55e-02 params: [3.6291603e-05 1.0576816e+00 2.7853630e+00]
Epoch [2200/6000] Loss: 3.53e-02 params: [3.0007723e-05 1.0572524e+00 2.8756745e+00]
Epoch [2400/6000] Loss: 3.51e-02 params: [3.1556119e-06

In [43]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=1.0655858e-06
fm=1.0486866
fp=4.5776043


In [44]:
# GEM 2N
# H_7, cGEM=50

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 50])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/H_7.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 7.09e-02 params: [0.6704456  0.85951006 0.8380866 ]
Epoch [400/6000] Loss: 3.04e-02 params: [0.5239458 1.0281659 1.0826951]
Epoch [600/6000] Loss: 2.70e-02 params: [0.455565  1.0679847 1.2905052]
Epoch [800/6000] Loss: 2.57e-02 params: [0.39881274 1.0694591  1.4847764 ]
Epoch [1000/6000] Loss: 2.53e-02 params: [0.33593798 1.0678356  1.6757944 ]
Epoch [1200/6000] Loss: 2.50e-02 params: [0.26635122 1.0658814  1.8641924 ]
Epoch [1400/6000] Loss: 2.47e-02 params: [0.19073687 1.0637679  2.050245  ]
Epoch [1600/6000] Loss: 2.47e-02 params: [0.10986291 1.061507   2.2341924 ]
Epoch [1800/6000] Loss: 2.42e-02 params: [0.02428535 1.0591661  2.4156687 ]
Epoch [2000/6000] Loss: 2.40e-02 params: [3.0850875e-05 1.0576987e+00 2.5952919e+00]
Epoch 02142: reducing learning rate of group 0 to 5.0000e-04.
Epoch [2200/6000] Loss: 2.38e-02 params: [2.1255128e-06 1.0570240e+00 2.7482035e+00]
Epoch [2400/6000] Loss: 2.37e-02 params: [4.1802923e-05 1.0565708e+00 2.8375311e+00]
Epoch [26

In [45]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=1.8087365e-05
fm=1.0503994
fp=4.193588


In [46]:
# GEM 2N
# E_8, cGEM=100

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 100])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/E_8.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 4.58e-02 params: [0.80533457 1.0338404  0.5343888 ]
Epoch [400/6000] Loss: 4.22e-02 params: [0.6840686 1.1326264 0.7499276]
Epoch [600/6000] Loss: 4.16e-02 params: [0.59175795 1.1461716  0.9468637 ]
Epoch [800/6000] Loss: 4.11e-02 params: [0.49460718 1.1423727  1.1412038 ]
Epoch [1000/6000] Loss: 4.08e-02 params: [0.38767305 1.1365101  1.3352242 ]
Epoch [1200/6000] Loss: 4.05e-02 params: [0.27254376 1.1301467  1.5288769 ]
Epoch [1400/6000] Loss: 4.03e-02 params: [0.15116332 1.1233045  1.7220223 ]
Epoch [1600/6000] Loss: 4.00e-02 params: [0.02526294 1.1165154  1.9144752 ]
Epoch [1800/6000] Loss: 3.98e-02 params: [1.0837401e-05 1.1135677e+00 2.1066046e+00]
Epoch [2000/6000] Loss: 3.98e-02 params: [3.4303976e-05 1.1125925e+00 2.2986355e+00]
Epoch 02003: reducing learning rate of group 0 to 5.0000e-04.
Epoch [2200/6000] Loss: 3.96e-02 params: [2.4935242e-05 1.1122041e+00 2.3966537e+00]
Epoch [2400/6000] Loss: 3.95e-02 params: [2.0022177e-05 1.1117043e+00 2.4930353e+0

In [47]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=3.1229443e-05
fm=1.1036727
fp=4.218063


In [48]:
# GEM 2N
# F_8, cGEM=100

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 100])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/F_8.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 1.17e-01 params: [0.3062629  0.41123444 0.34378684]
Epoch [400/6000] Loss: 9.25e-02 params: [0.0932148 0.693674  0.6470136]
Epoch [600/6000] Loss: 7.30e-02 params: [5.2534469e-05 9.4881201e-01 9.7098666e-01]
Epoch [800/6000] Loss: 6.69e-02 params: [3.2159252e-05 1.0833205e+00 1.2493107e+00]
Epoch [1000/6000] Loss: 6.59e-02 params: [1.07773385e-05 1.12114048e+00 1.48233485e+00]
Epoch [1200/6000] Loss: 6.55e-02 params: [3.9134106e-06 1.1274571e+00 1.6980042e+00]
Epoch [1400/6000] Loss: 6.52e-02 params: [7.6747310e-06 1.1275240e+00 1.9066929e+00]
Epoch [1600/6000] Loss: 6.49e-02 params: [7.4950331e-06 1.1266627e+00 2.1112528e+00]
Epoch [1800/6000] Loss: 6.47e-02 params: [7.4918818e-05 1.1257035e+00 2.3128216e+00]
Epoch [2000/6000] Loss: 6.44e-02 params: [8.2767343e-05 1.1247522e+00 2.5120585e+00]
Epoch [2200/6000] Loss: 6.41e-02 params: [1.4387340e-04 1.1236724e+00 2.7094104e+00]
Epoch [2400/6000] Loss: 6.39e-02 params: [1.3834666e-05 1.1227016e+00 2.9051847e+00]
Ep

In [49]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=9.982588e-05
fm=1.1073649
fp=6.2910576


In [50]:
# GEM 2N
# G_8, cGEM=100

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 100])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/G_8.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 5.92e-02 params: [0.01467705 1.1065853  0.8402527 ]
Epoch [400/6000] Loss: 5.57e-02 params: [1.7067880e-04 1.1184342e+00 1.0519190e+00]
Epoch [600/6000] Loss: 5.49e-02 params: [1.2844204e-04 1.1183609e+00 1.2579942e+00]
Epoch [800/6000] Loss: 5.46e-02 params: [2.4602386e-05 1.1179847e+00 1.4604605e+00]
Epoch [1000/6000] Loss: 5.44e-02 params: [5.3640688e-05 1.1173435e+00 1.6604837e+00]
Epoch [1200/6000] Loss: 5.42e-02 params: [1.0796789e-04 1.1162800e+00 1.8588300e+00]
Epoch [1400/6000] Loss: 5.40e-02 params: [1.5854687e-04 1.1155119e+00 2.0557809e+00]
Epoch [1600/6000] Loss: 5.38e-02 params: [3.8688224e-05 1.1146864e+00 2.2515926e+00]
Epoch [1800/6000] Loss: 5.36e-02 params: [5.4094635e-06 1.1139449e+00 2.4464216e+00]
Epoch [2000/6000] Loss: 5.43e-02 params: [1.1198459e-04 1.1129601e+00 2.6404362e+00]
Epoch [2200/6000] Loss: 5.33e-02 params: [9.0629990e-05 1.1123018e+00 2.8335981e+00]
Epoch [2400/6000] Loss: 5.31e-02 params: [3.2960204e-05 1.1115226e+00 3.026015

In [51]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=1.1365548e-05
fm=1.1008618
fp=5.4660335


In [52]:
# GEM 2N
# H_8, cGEM=100

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 100])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/H_8.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 6.72e-02 params: [3.115518e-04 7.714279e-01 7.795229e-01]
Epoch [400/6000] Loss: 5.07e-02 params: [1.3273529e-07 9.6009928e-01 1.0139719e+00]
Epoch [600/6000] Loss: 4.62e-02 params: [1.3499008e-04 1.0776312e+00 1.2478522e+00]
Epoch [800/6000] Loss: 4.55e-02 params: [2.6904481e-05 1.1160524e+00 1.4575315e+00]
Epoch [1000/6000] Loss: 4.53e-02 params: [5.3923883e-05 1.1235467e+00 1.6574550e+00]
Epoch [1200/6000] Loss: 4.50e-02 params: [1.0111622e-05 1.1240056e+00 1.8545651e+00]
Epoch [1400/6000] Loss: 4.48e-02 params: [4.4728629e-05 1.1233829e+00 2.0504527e+00]
Epoch [1600/6000] Loss: 4.46e-02 params: [1.07816995e-05 1.12264001e+00 2.24542952e+00]
Epoch 01795: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1800/6000] Loss: 4.44e-02 params: [1.1441890e-05 1.1217194e+00 2.4377387e+00]
Epoch [2000/6000] Loss: 4.43e-02 params: [2.5083376e-05 1.1214164e+00 2.5347388e+00]
Epoch [2200/6000] Loss: 4.42e-02 params: [2.0912010e-05 1.1209447e+00 2.6318781e+00]
Epoch [

In [53]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=6.9107773e-07
fm=1.1142932
fp=4.4752164


In [54]:
# GEM 2N
# E_9, cGEM=200

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 200])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/E_9.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 7.92e-02 params: [0.456466  0.728376  0.5648284]
Epoch [400/6000] Loss: 7.09e-02 params: [0.26792398 0.94816625 0.8150009 ]
Epoch [600/6000] Loss: 6.87e-02 params: [0.13600668 1.1036136  1.0642304 ]
Epoch [800/6000] Loss: 6.78e-02 params: [0.05540938 1.1735176  1.2903502 ]
Epoch [1000/6000] Loss: 6.74e-02 params: [0.00230739 1.1928709  1.502666  ]
Epoch [1200/6000] Loss: 6.73e-02 params: [3.2276745e-05 1.1959909e+00 1.7088273e+00]
Epoch [1400/6000] Loss: 6.72e-02 params: [4.4926284e-05 1.1965617e+00 1.9123925e+00]
Epoch [1600/6000] Loss: 6.71e-02 params: [1.6834974e-05 1.1962538e+00 2.1142242e+00]
Epoch [1800/6000] Loss: 6.70e-02 params: [4.5587516e-05 1.1957932e+00 2.3148763e+00]
Epoch 01999: reducing learning rate of group 0 to 5.0000e-04.
Epoch [2000/6000] Loss: 6.70e-02 params: [3.5722398e-05 1.1950691e+00 2.5146537e+00]
Epoch [2200/6000] Loss: 6.69e-02 params: [5.1427774e-05 1.1949090e+00 2.6142838e+00]
Epoch [2400/6000] Loss: 6.69e-02 params: [5.420274e-05 

In [55]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=1.5959307e-09
fm=1.1927582
fp=3.2976458


In [56]:
# GEM 2N
# F_9, cGEM=200

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 200])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/F_9.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 1.05e-01 params: [0.344596   0.22868241 0.6095434 ]
Epoch [400/6000] Loss: 9.50e-02 params: [0.16547008 0.2552732  0.6363203 ]
Epoch [600/6000] Loss: 9.24e-02 params: [0.00086441 0.29471874 0.6770683 ]
Epoch [800/6000] Loss: 9.10e-02 params: [4.2807183e-05 3.4676793e-01 7.3243147e-01]
Epoch [1000/6000] Loss: 8.97e-02 params: [2.6601123e-05 4.1180757e-01 8.0394119e-01]
Epoch [1200/6000] Loss: 8.85e-02 params: [1.9781190e-04 4.9051711e-01 8.9453155e-01]
Epoch [1400/6000] Loss: 8.72e-02 params: [1.6816859e-05 5.8335918e-01 1.0078924e+00]
Epoch [1600/6000] Loss: 8.56e-02 params: [1.0746986e-05 6.8935317e-01 1.1474079e+00]
Epoch [1800/6000] Loss: 8.39e-02 params: [1.2303647e-04 8.0415493e-01 1.3140090e+00]
Epoch [2000/6000] Loss: 8.23e-02 params: [9.6950444e-06 9.1863126e-01 1.5033554e+00]
Epoch [2200/6000] Loss: 8.11e-02 params: [1.6591845e-04 1.0208917e+00 1.7060133e+00]
Epoch [2400/6000] Loss: 8.04e-02 params: [6.5680259e-05 1.1016396e+00 1.9118071e+00]
Epoch [2600

In [57]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=6.832816e-10
fm=1.2320665
fp=3.1543045


In [58]:
# GEM 2N
# G_9, cGEM=200

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 200])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/G_9.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 6.50e-02 params: [0.24874143 0.2140818  0.2816104 ]
Epoch [400/6000] Loss: 5.75e-02 params: [0.03473947 0.47155347 0.54635394]
Epoch [600/6000] Loss: 5.35e-02 params: [1.2779509e-04 7.2484499e-01 8.3889824e-01]
Epoch [800/6000] Loss: 5.09e-02 params: [3.5089131e-05 9.4259596e-01 1.1312717e+00]
Epoch [1000/6000] Loss: 4.98e-02 params: [1.9683297e-05 1.0929873e+00 1.4003252e+00]
Epoch [1200/6000] Loss: 4.95e-02 params: [9.5363957e-06 1.1750771e+00 1.6420807e+00]
Epoch 01366: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1400/6000] Loss: 4.94e-02 params: [1.8286368e-05 1.2089437e+00 1.8475019e+00]
Epoch [1600/6000] Loss: 4.93e-02 params: [6.9660741e-06 1.2179531e+00 1.9547528e+00]
Epoch [1800/6000] Loss: 4.93e-02 params: [1.6777790e-08 1.2230169e+00 2.0596027e+00]
Epoch [2000/6000] Loss: 4.93e-02 params: [2.7962242e-06 1.2255871e+00 2.1629024e+00]
Epoch [2200/6000] Loss: 4.93e-02 params: [1.6177190e-05 1.2266839e+00 2.2651680e+00]
Epoch [2400/6000] Loss: 4

In [59]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=8.801196e-10
fm=1.2268244
fp=2.5080142


In [60]:
# GEM 2N
# H_9, cGEM=200

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 200])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/H_9.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 6.75e-02 params: [0.63290405 0.66577494 0.7333045 ]
Epoch [400/6000] Loss: 6.22e-02 params: [0.4362826 0.8587542 0.9527434]
Epoch [600/6000] Loss: 5.86e-02 params: [0.28569385 1.0266832  1.1939393 ]
Epoch [800/6000] Loss: 5.69e-02 params: [0.1872314 1.1312431 1.4261549]
Epoch [1000/6000] Loss: 5.64e-02 params: [0.12884507 1.1772256  1.643751  ]
Epoch [1200/6000] Loss: 5.61e-02 params: [0.09007017 1.190358   1.8526031 ]
Epoch [1400/6000] Loss: 5.59e-02 params: [0.0575544 1.1902305 2.0580626]
Epoch [1600/6000] Loss: 5.57e-02 params: [0.02552339 1.1857156  2.2623765 ]
Epoch [1800/6000] Loss: 5.55e-02 params: [2.0690961e-05 1.1804650e+00 2.4661593e+00]
Epoch 01848: reducing learning rate of group 0 to 5.0000e-04.
Epoch [2000/6000] Loss: 5.54e-02 params: [2.1467742e-05 1.1793941e+00 2.5928941e+00]
Epoch [2200/6000] Loss: 5.54e-02 params: [2.3610690e-05 1.1790680e+00 2.6943555e+00]
Epoch [2400/6000] Loss: 5.53e-02 params: [2.5560954e-05 1.1792120e+00 2.7955923e+00]
Epo

In [61]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=3.927663e-09
fm=1.183826
fp=3.958254


In [62]:
# GEM 2N
# E_1_0, cGEM=400

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 400])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/E_1_0.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 9.31e-02 params: [0.17479105 0.37948513 0.3964633 ]
Epoch [400/6000] Loss: 6.99e-02 params: [0.07122979 0.38805473 0.41266134]
Epoch [600/6000] Loss: 6.80e-02 params: [4.1940330e-05 4.0080038e-01 4.3652922e-01]
Epoch [800/6000] Loss: 6.71e-02 params: [7.1505725e-05 4.2027915e-01 4.6952626e-01]
Epoch [1000/6000] Loss: 6.64e-02 params: [1.0625911e-04 4.4538140e-01 5.1096416e-01]
Epoch [1200/6000] Loss: 6.60e-02 params: [1.1613925e-04 4.7508124e-01 5.6038129e-01]
Epoch [1400/6000] Loss: 6.58e-02 params: [1.3714764e-04 5.0886500e-01 6.1799812e-01]
Epoch 01482: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1600/6000] Loss: 6.57e-02 params: [1.2356668e-07 5.3527534e-01 6.6426611e-01]
Epoch [1800/6000] Loss: 6.56e-02 params: [2.5844914e-05 5.5625743e-01 7.0157707e-01]
Epoch [2000/6000] Loss: 6.55e-02 params: [4.3139615e-05 5.7940686e-01 7.4327463e-01]
Epoch [2200/6000] Loss: 6.55e-02 params: [8.1517748e-05 6.0473174e-01 7.8949827e-01]
Epoch [2400/6000] Loss: 6

In [63]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.0012076304
fm=1.1070297
fp=1.9487678


In [64]:
# GEM 2N
# F_1_0, cGEM=400

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 400])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/F_1_0.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 9.23e-02 params: [0.6259658 0.8211865 1.1000136]
Epoch [400/6000] Loss: 9.03e-02 params: [0.44858184 1.0345337  1.3316716 ]
Epoch [600/6000] Loss: 8.57e-02 params: [0.32183766 1.2018756  1.5625621 ]
Epoch [800/6000] Loss: 8.45e-02 params: [0.24533188 1.3174976  1.7781011 ]
Epoch [1000/6000] Loss: 8.39e-02 params: [0.21281305 1.3955349  1.9830164 ]
Epoch [1200/6000] Loss: 8.38e-02 params: [0.2139319 1.4478629 2.1830149]
Epoch 01248: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1400/6000] Loss: 8.37e-02 params: [0.22731966 1.4724258  2.3063123 ]
Epoch [1600/6000] Loss: 8.36e-02 params: [0.2440103 1.4897121 2.4050653]
Epoch [1800/6000] Loss: 8.36e-02 params: [0.2651378 1.5059513 2.5038657]
Epoch [2000/6000] Loss: 8.35e-02 params: [0.28990108 1.5216918  2.6027393 ]
Epoch [2200/6000] Loss: 8.43e-02 params: [0.3174419 1.5372692 2.7017047]
Epoch 02229: reducing learning rate of group 0 to 2.5000e-04.
Epoch 02304: reducing learning rate of group 0 to 1.2500e-0

In [65]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.3324318
fm=1.545337
fp=2.7516282


In [66]:
# GEM 2N
# G_1_0, cGEM=400

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 400])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/G_1_0.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 8.31e-02 params: [0.03957856 0.98754776 1.0067217 ]
Epoch [400/6000] Loss: 6.22e-02 params: [4.2567823e-05 1.0058291e+00 1.0713180e+00]
Epoch [600/6000] Loss: 5.98e-02 params: [1.0317544e-05 1.0444534e+00 1.1680903e+00]
Epoch [800/6000] Loss: 5.84e-02 params: [5.6548710e-05 1.0844729e+00 1.2843903e+00]
Epoch [1000/6000] Loss: 5.79e-02 params: [1.4897223e-04 1.1208252e+00 1.4149680e+00]
Epoch [1200/6000] Loss: 5.75e-02 params: [0.01254778 1.1524143  1.5575848 ]
Epoch 01373: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1400/6000] Loss: 5.73e-02 params: [0.05028008 1.1823404  1.7005999 ]
Epoch [1600/6000] Loss: 5.72e-02 params: [0.07652012 1.2005979  1.781318  ]
Epoch [1800/6000] Loss: 5.71e-02 params: [0.10637888 1.2216295  1.8656718 ]
Epoch [2000/6000] Loss: 5.71e-02 params: [0.14050391 1.2456528  1.9529779 ]
Epoch [2200/6000] Loss: 5.70e-02 params: [0.17922157 1.273153   2.0427566 ]
Epoch [2400/6000] Loss: 5.69e-02 params: [0.2228502 1.304501  2.134596

In [67]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.57977897
fm=1.572148
fp=2.6910331


In [68]:
# GEM 2N
# H_1_0, cGEM=400

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 400])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/H_1_0.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 8.53e-02 params: [0.16768226 0.86421543 0.5604614 ]
Epoch [400/6000] Loss: 8.11e-02 params: [0.02327862 1.0674002  0.8018701 ]
Epoch [600/6000] Loss: 7.81e-02 params: [2.7326887e-04 1.1934761e+00 1.0277101e+00]
Epoch [800/6000] Loss: 7.55e-02 params: [0.05352505 1.2761959  1.242621  ]
Epoch [1000/6000] Loss: 7.37e-02 params: [0.17446132 1.3623489  1.4517132 ]
Epoch [1200/6000] Loss: 7.34e-02 params: [0.32026502 1.4599266  1.659073  ]
Epoch [1400/6000] Loss: 7.31e-02 params: [0.46641466 1.5516701  1.8648331 ]
Epoch [1600/6000] Loss: 7.29e-02 params: [0.5980778 1.6299384 2.0686812]
Epoch 01669: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1800/6000] Loss: 7.28e-02 params: [0.6747599 1.6740144 2.2054224]
Epoch [2000/6000] Loss: 7.27e-02 params: [0.7268355 1.7032079 2.3054628]
Epoch [2200/6000] Loss: 7.26e-02 params: [0.7739929 1.7295457 2.4053094]
Epoch [2400/6000] Loss: 7.24e-02 params: [0.8153701 1.7527975 2.5050118]
Epoch [2600/6000] Loss: 7.23e-02 par

In [69]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.9364492
fm=1.8301921
fp=3.34505


In [72]:
# GEM 2N
# E_1_1, cGEM=800

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 800])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/E_1_1.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 1.24e-01 params: [0.9286952  0.63773704 0.97067446]
Epoch [400/6000] Loss: 7.23e-02 params: [0.8352566 0.6380765 0.9714807]
Epoch [600/6000] Loss: 6.67e-02 params: [0.71847856 0.6388872  0.97284335]
Epoch [800/6000] Loss: 6.53e-02 params: [0.6039174 0.6399062 0.9746602]
Epoch [1000/6000] Loss: 6.44e-02 params: [0.50334483 0.64094335 0.97687274]
Epoch [1200/6000] Loss: 6.37e-02 params: [0.42065483 0.6419512  0.97947043]
Epoch [1400/6000] Loss: 6.32e-02 params: [0.35719404 0.6428941  0.98246527]
Epoch [1600/6000] Loss: 6.29e-02 params: [0.31204763 0.6437693  0.98588496]
Epoch [1800/6000] Loss: 6.27e-02 params: [0.28198704 0.64456654 0.9897645 ]
Epoch [2000/6000] Loss: 6.26e-02 params: [0.26292562 0.6452806  0.9941548 ]
Epoch [2200/6000] Loss: 6.25e-02 params: [0.2515051  0.64594275 0.99911284]
Epoch [2400/6000] Loss: 6.25e-02 params: [0.24543737 0.6466004  1.0046887 ]
Epoch [2600/6000] Loss: 6.24e-02 params: [0.24295107 0.64730036 1.0109292 ]
Epoch [2800/6000] Loss

In [73]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.23262458
fm=0.66367507
fp=1.1221453


In [74]:
# GEM 2N
# F_1_1, cGEM=800

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 800])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/F_1_1.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 9.11e-02 params: [0.17063612 0.7386125  0.34696758]
Epoch [400/6000] Loss: 8.15e-02 params: [0.14184369 0.7268779  0.37928057]
Epoch [600/6000] Loss: 8.01e-02 params: [0.10472299 0.7159409  0.42738083]
Epoch [800/6000] Loss: 7.88e-02 params: [0.07531524 0.7035174  0.48551378]
Epoch [1000/6000] Loss: 7.78e-02 params: [0.06236503 0.6899714  0.552377  ]
Epoch [1200/6000] Loss: 7.72e-02 params: [0.06626911 0.6799857  0.63207185]
Epoch [1400/6000] Loss: 7.68e-02 params: [0.07084955 0.67031574 0.7205501 ]
Epoch 01500: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1600/6000] Loss: 7.67e-02 params: [0.07271408 0.66405344 0.7941748 ]
Epoch [1800/6000] Loss: 7.66e-02 params: [0.07311676 0.657624   0.8465374 ]
Epoch [2000/6000] Loss: 7.66e-02 params: [0.07241961 0.65090734 0.9036463 ]
Epoch [2200/6000] Loss: 7.65e-02 params: [0.07063818 0.64393276 0.9653252 ]
Epoch [2400/6000] Loss: 7.65e-02 params: [0.06783809 0.63666826 1.0313339 ]
Epoch 02486: reducing learning

In [75]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.06435904
fm=0.62953514
fp=1.0969592


In [76]:
# GEM 2N
# G_1_1, cGEM=800

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 800])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/G_1_1.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 5.96e-02 params: [0.92413116 0.45532396 0.5568676 ]
Epoch [400/6000] Loss: 5.12e-02 params: [0.863568   0.45720732 0.5587426 ]
Epoch [600/6000] Loss: 4.99e-02 params: [0.7888592  0.45979068 0.5618728 ]
Epoch [800/6000] Loss: 4.84e-02 params: [0.70791817 0.4628213  0.56598634]
Epoch [1000/6000] Loss: 4.71e-02 params: [0.6266822 0.4660703 0.5706814]
Epoch [1200/6000] Loss: 4.68e-02 params: [0.54785496 0.4697924  0.57610255]
Epoch [1400/6000] Loss: 4.67e-02 params: [0.47181153 0.47381616 0.58238   ]
Epoch [1600/6000] Loss: 4.68e-02 params: [0.3999746  0.47796896 0.589575  ]
Epoch [1800/6000] Loss: 4.65e-02 params: [0.333754   0.48214602 0.5977476 ]
Epoch [2000/6000] Loss: 4.65e-02 params: [0.2742442  0.48625705 0.6069667 ]
Epoch 02042: reducing learning rate of group 0 to 5.0000e-04.
Epoch 02104: reducing learning rate of group 0 to 2.5000e-04.
Epoch 02167: reducing learning rate of group 0 to 1.2500e-04.
Epoch [2200/6000] Loss: 4.65e-02 params: [0.2487457  0.488154

In [77]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.24624155
fm=0.48834476
fp=0.61221725


In [78]:
# GEM 2N
# H_1_1, cGEM=800

known_param_4N = np.array([0.04754016, 1.06978193, 0.02534762, 0.00800557, 0.00394785, 800])
file_G1, file_G2, file_P, file_D, file_G1_pred, file_G2_pred, file_P_pred, file_D_pred = drug_dose_param(pd.read_csv('csvs/H_1_1.csv'),known_param_4N,t)


Epoch [200/6000] Loss: 7.47e-02 params: [0.41937447 0.97533095 0.31205902]
Epoch [400/6000] Loss: 6.61e-02 params: [0.36295927 0.98036677 0.3168409 ]
Epoch [600/6000] Loss: 6.42e-02 params: [0.30480108 0.9862511  0.3236208 ]
Epoch [800/6000] Loss: 6.09e-02 params: [0.26117033 0.9934751  0.3320907 ]
Epoch [1000/6000] Loss: 5.73e-02 params: [0.25049317 1.0026685  0.34237757]
Epoch [1200/6000] Loss: 5.64e-02 params: [0.27143803 1.0131603  0.35475716]
Epoch [1400/6000] Loss: 5.61e-02 params: [0.2978678  1.0255259  0.36920884]
Epoch [1600/6000] Loss: 5.63e-02 params: [0.31885204 1.0402637  0.38584784]
Epoch 01602: reducing learning rate of group 0 to 5.0000e-04.
Epoch [1800/6000] Loss: 5.59e-02 params: [0.32739615 1.048919   0.39545423]
Epoch [2000/6000] Loss: 5.58e-02 params: [0.33539507 1.0586244  0.40617102]
Epoch [2200/6000] Loss: 5.58e-02 params: [0.34293333 1.069599   0.41823554]
Epoch [2400/6000] Loss: 5.57e-02 params: [0.3502186 1.0819548 0.4317673]
Epoch [2600/6000] Loss: 5.57e-02 

In [82]:
# PINN params
loss_param = pd.read_csv('loss_param_1e.csv')
fg = loss_param['fg']; fm = loss_param['fm']; fp = loss_param['fp']; 
fg_param = fg[len(fg)-1]; fm_param = fm[len(fm)-1]; fp_param = fp[len(fp)-1];

print('fg='+str(fg_param))
print('fm='+str(fm_param))
print('fp='+str(fp_param))


fg=0.48527053
fm=1.398501
fp=0.76900977


In [83]:
import nbformat
import re
import numpy as np

# Path to the current notebook
notebook_path = "GEM_4n.ipynb"  # Replace with the actual filename of your notebook

# Lists to store the extracted values
fg_values, fm_values, fp_values = [], [], []

# Load the notebook
with open(notebook_path, "r", encoding="utf-8") as f:
    notebook = nbformat.read(f, as_version=4)

# Iterate over each cell in the notebook
for cell in notebook.cells:
    # Check if the cell contains output
    if cell.cell_type == "code" and "outputs" in cell:
        for output in cell.outputs:
            # Ensure the output is text type and check for pattern
            if output.output_type == "stream" and "text" in output:
                output_text = output["text"]

                # Use regex to extract fg, fm, and fp values
                fg_match = re.search(r"fg=([\d.eE+-]+)", output_text)
                fm_match = re.search(r"fm=([\d.eE+-]+)", output_text)
                fp_match = re.search(r"fp=([\d.eE+-]+)", output_text)

                # Append values if all three are found
                if fg_match:
                    fg_values.append(float(fg_match.group(1)))
                if fm_match:
                    fm_values.append(float(fm_match.group(1)))
                if fp_match:
                    fp_values.append(float(fp_match.group(1)))

# Convert lists to separate matrices
fg_matrix = np.array(fg_values).reshape(-1, 1)
fm_matrix = np.array(fm_values).reshape(-1, 1)
fp_matrix = np.array(fp_values).reshape(-1, 1)

# Display the matrices
print("FG Matrix:")
print(fg_matrix)
print("\nFM Matrix:")
print(fm_matrix)
print("\nFP Matrix:")
print(fp_matrix)


FG Matrix:
[[4.8942767e-02]
 [5.8532105e-06]
 [1.2257206e-06]
 [2.1821547e-01]
 [4.5898990e-01]
 [1.3935164e-01]
 [2.0507825e-06]
 [8.5874290e-03]
 [2.7072486e-01]
 [1.5408150e-06]
 [5.5031530e-06]
 [2.6773840e-05]
 [3.3405960e-05]
 [1.6821685e-05]
 [1.4426978e-05]
 [2.6213222e-05]
 [4.7238440e-05]
 [2.4882750e-05]
 [1.0655858e-06]
 [1.8087365e-05]
 [3.1229443e-05]
 [9.9825880e-05]
 [1.1365548e-05]
 [6.9107773e-07]
 [1.5959307e-09]
 [6.8328160e-10]
 [8.8011960e-10]
 [3.9276630e-09]
 [1.2076304e-03]
 [3.3243180e-01]
 [5.7977897e-01]
 [9.3644920e-01]
 [2.3262458e-01]
 [6.4359040e-02]
 [2.4624155e-01]
 [4.8527053e-01]]

FM Matrix:
[[3.3381486e-05]
 [2.8474462e-01]
 [6.8456426e-02]
 [1.4673647e-01]
 [6.6858166e-01]
 [6.7333984e-01]
 [2.3787631e-01]
 [2.0006588e-01]
 [8.6827725e-01]
 [9.0340980e-01]
 [8.2989380e-01]
 [8.9286816e-01]
 [1.0007280e+00]
 [9.9840050e-01]
 [9.9280643e-01]
 [1.0128875e+00]
 [1.0342630e+00]
 [1.0428274e+00]
 [1.0486866e+00]
 [1.0503994e+00]
 [1.1036727e+00]
 [1.107

In [84]:
fg_matrix_2d = fg_matrix.reshape(9,4)
fg_matrix_2d_T = fg_matrix_2d.T
m_fg_matrix_2d_T = np.insert(fg_matrix_2d_T, 0, 0, axis=1)

fm_matrix_2d = fm_matrix.reshape(9,4)
fm_matrix_2d_T = fm_matrix_2d.T
m_fm_matrix_2d_T = np.insert(fm_matrix_2d_T, 0, 0, axis=1)

fp_matrix_2d = fp_matrix.reshape(9,4)
fp_matrix_2d_T = fp_matrix_2d.T
m_fp_matrix_2d_T = np.insert(fp_matrix_2d_T, 0, 0, axis=1)

In [85]:


print(m_fg_matrix_2d_T)

[[0.0000000e+00 4.8942767e-02 4.5898990e-01 2.7072486e-01 3.3405960e-05
  4.7238440e-05 3.1229443e-05 1.5959307e-09 1.2076304e-03 2.3262458e-01]
 [0.0000000e+00 5.8532105e-06 1.3935164e-01 1.5408150e-06 1.6821685e-05
  2.4882750e-05 9.9825880e-05 6.8328160e-10 3.3243180e-01 6.4359040e-02]
 [0.0000000e+00 1.2257206e-06 2.0507825e-06 5.5031530e-06 1.4426978e-05
  1.0655858e-06 1.1365548e-05 8.8011960e-10 5.7977897e-01 2.4624155e-01]
 [0.0000000e+00 2.1821547e-01 8.5874290e-03 2.6773840e-05 2.6213222e-05
  1.8087365e-05 6.9107773e-07 3.9276630e-09 9.3644920e-01 4.8527053e-01]]


In [86]:


print(m_fm_matrix_2d_T)

[[0.0000000e+00 3.3381486e-05 6.6858166e-01 8.6827725e-01 1.0007280e+00
  1.0342630e+00 1.1036727e+00 1.1927582e+00 1.1070297e+00 6.6367507e-01]
 [0.0000000e+00 2.8474462e-01 6.7333984e-01 9.0340980e-01 9.9840050e-01
  1.0428274e+00 1.1073649e+00 1.2320665e+00 1.5453370e+00 6.2953514e-01]
 [0.0000000e+00 6.8456426e-02 2.3787631e-01 8.2989380e-01 9.9280643e-01
  1.0486866e+00 1.1008618e+00 1.2268244e+00 1.5721480e+00 4.8834476e-01]
 [0.0000000e+00 1.4673647e-01 2.0006588e-01 8.9286816e-01 1.0128875e+00
  1.0503994e+00 1.1142932e+00 1.1838260e+00 1.8301921e+00 1.3985010e+00]]


In [87]:


print(m_fp_matrix_2d_T)

[[0.         0.93288296 2.8191557  3.2862406  4.280553   5.092544
  4.218063   3.2976458  1.9487678  1.1221453 ]
 [0.         1.0649403  2.0293932  2.5255768  4.1156373  6.0146546
  6.2910576  3.1543045  2.7516282  1.0969592 ]
 [0.         0.9525176  0.6392054  2.6088068  4.2057157  4.5776043
  5.4660335  2.5080142  2.6910331  0.61221725]
 [0.         1.4883664  1.0710055  2.7416446  4.0853634  4.193588
  4.4752164  3.958254   3.34505    0.76900977]]


In [90]:
# median fg 3
fg_median_3 = np.median(m_fg_matrix_2d_T[:,1])
print(fg_median_3)

# median fm 3
fm_median_3 = np.median(m_fm_matrix_2d_T[:,1])
print(fm_median_3)

# median fp 3
fp_median_3 = np.median(m_fp_matrix_2d_T[:,1])
print(fp_median_3)


0.02447431010525
0.10759644800000001
1.00872895
